In [1]:
!pip install diffusers transformers accelerate torch pillow imageio[ffmpeg] --quiet

In [1]:
from diffusers import DiffusionPipeline
import torch

pipe = DiffusionPipeline.from_pretrained(
    "cerspense/zeroscope_v2_576w",
    torch_dtype=torch.float16
).to("cuda")

print("ZeroScope Loaded ✔️")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

scheduler_config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

...

unet/diffusion_pytorch_model.bin:   0%|          | 0.00/2.82G [00:00<?, ?B/s]

vae/diffusion_pytorch_model.bin:   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--cerspense--zeroscope_v2_576w/snapshots/6963642a64dbefa93663d1ecebb4ceda2d9ecb28/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--cerspense--zeroscope_v2_576w/snapshots/6963642a64dbefa93663d1ecebb4ceda2d9ecb28/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
`torch_dtype` is deprecated! Use `dtype` instead!
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--cerspense--zeroscope_v2_576w/snapshots/6963642a64dbefa93663d1ecebb4ceda2d9ecb28/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--cerspense--zeroscope_v2_576w/snapshots/6963642a64dbefa93663d1ecebb4ceda2d9ecb28/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
The TextToVideoSDPipeline has been deprecated and wi

ZeroScope Loaded ✔️


In [2]:
prompt = "a cat riding a bike in a futuristic city, cinematic lighting, 4k style"

video_frames = pipe(prompt, num_frames=24).frames


  0%|          | 0/50 [00:00<?, ?it/s]

In [6]:
import imageio
import numpy as np

fixed_frames = []

# Corrected loop: iterate over the frames within the batch
for frame in video_frames[0]:
    # At this point, `frame` should be a NumPy array of shape (C, H, W)
    # The kernel state for `video_frames` indicates it's already a numpy array.
    # So, `hasattr(frame, "cpu")` will be False.

    # If shape is (3, H, W) → convert to (H, W, 3)
    if frame.shape[0] == 3:
        frame = np.transpose(frame, (1, 2, 0))

    # Convert float32 0–1 → uint8 0–255
    frame = (frame * 255).clip(0, 255).astype(np.uint8)

    fixed_frames.append(frame)

# Save video
video_path = "output_fixed.mp4"
imageio.mimsave(video_path, fixed_frames, fps=12)

video_path

'output_fixed.mp4'

In [7]:
from IPython.display import HTML
from base64 import b64encode

mp4 = open(video_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML(f"""
<video width=512 controls>
  <source src="{data_url}" type="video/mp4">
</video>
""")
